### Notebook to generate DroneRF Features

In [1]:
import os
import numpy as np
from numpy import sum,isrealobj,sqrt
from numpy.random import standard_normal
from sklearn.model_selection import train_test_split
from spafe.features.lfcc import lfcc
import spafe.utils.vis as vis
from scipy.signal import get_window
import scipy.fftpack as fft
from scipy import signal
import matplotlib.pyplot as plt
from datetime import date
from tqdm import tqdm

# from loading_functions import *
from file_paths import *
from feat_gen_functions import *

import importlib
import pandas as pd

In [2]:
import feat_gen_functions
importlib.reload(feat_gen_functions)
from feat_gen_functions import *

In [ ]:
# spark.stop()

In [ ]:
# from pyspark.sql import SparkSession

# from pyspark.sql.types import *


# spark = SparkSession.builder \
#     .appName("DroneRFDistributedProcessing") \
#     .master("local[*]") \
#     .config("spark.driver.memory", "8g") \
#     .config("spark.executor.memory", "8g") \
#     .config("spark.memory.fraction", "0.8") \
#     .config("spark.sql.shuffle.partitions", "200") \
#     .config("spark.default.parallelism", "100") \
#     .config("spark.driver.maxResultSize", "4g") \
#     .getOrCreate()

# # Broadcast frequently used constants to all workers
# fs = spark.sparkContext.broadcast(40e6)

In [5]:
def load_dronerf_raw(main_folder, t_seg):
    import os
    import numpy as np
    import time
    from datetime import datetime
    import pandas as pd
    from pyspark.sql.functions import udf, lit, col
    from pyspark.sql.types import ArrayType, FloatType, IntegerType, StructType, StructField, StringType, LongType

    print(f"Started processing at {datetime.now().strftime('%H:%M:%S')}")
    start_time_total = time.time()

    # Get file lists
    high_freq_files = os.listdir(main_folder+'High/')
    low_freq_files = os.listdir(main_folder+'Low/')
    
    high_freq_files.sort()
    low_freq_files.sort()
    
    print(f"Found {len(high_freq_files)} high frequency files and {len(low_freq_files)} low frequency files")
    
    # Define the segment length
    len_seg = int((t_seg/1e3) * 40e6)  # 40 MHz
    
    # Process files in smaller batches to avoid timeout
    batch_size = 5  # Process 5 files at a time
    total_batches = (len(high_freq_files) + batch_size - 1) // batch_size
    
    # Arrays to store results
    all_features = []
    all_y = []
    all_y4 = []
    all_y10 = []
    
    # Track processing statistics
    timing_stats = []
    successful_files = 0
    
    # Process files in batches
    for batch in range(total_batches):
        start_idx = batch * batch_size
        end_idx = min((batch + 1) * batch_size, len(high_freq_files))
        
        print(f"\nProcessing batch {batch+1}/{total_batches} (files {start_idx+1}-{end_idx})")
        
        # Process each file in the batch
        for i in range(start_idx, end_idx):
            file_start_time = time.time()
            
            high_file = high_freq_files[i]
            low_file = low_freq_files[i]
            
            try:
                # Get file sizes
                high_path = main_folder + 'High/' + high_file
                low_path = main_folder + 'Low/' + low_file
                file_size_high = os.path.getsize(high_path)
                file_size_low = os.path.getsize(low_path)
                
                print(f"Processing file {i+1}/{len(high_freq_files)}: {high_file} ({file_size_high/1024:.1f}KB) & {low_file} ({file_size_low/1024:.1f}KB)")
                
                # Load RF data
                load_start = time.time()
                rf_data_h = pd.read_csv(high_path, header=None).values.flatten()
                rf_data_l = pd.read_csv(low_path, header=None).values.flatten()
                load_time = time.time() - load_start
                print(f"  - Load time: {load_time:.2f}s")
                
                if len(rf_data_h) != len(rf_data_l):
                    print(f'  - Different lengths: {i}, file name: {low_file}')
                    continue
                    
                if int(high_file[:5]) != int(low_file[:5]):
                    print(f"  - File labels do not match: {high_file} vs {low_file}")
                    continue
                    
                # Stack features
                stack_start = time.time()
                rf_sig = np.vstack((rf_data_h, rf_data_l))
                stack_time = time.time() - stack_start
                
                # Calculate segments
                n_segs = len(rf_data_h) // len_seg
                n_keep = n_segs * len_seg
                
                if n_segs == 0:
                    print("  - No segments created (segment length too large)")
                    continue
                    
                # Split the data
                split_start = time.time()
                rf_sig_segments = np.split(rf_sig[:, :n_keep], n_segs, axis=1)
                split_time = time.time() - split_start
                
                # Create labels
                y_rep = [int(low_file[0])] * n_segs
                y4_rep = [int(low_file[:3])] * n_segs
                y10_rep = [int(low_file[:5])] * n_segs
                
                # Add segments to our result arrays
                all_features.extend(rf_sig_segments)
                all_y.extend(y_rep)
                all_y4.extend(y4_rep)
                all_y10.extend(y10_rep)
                
                successful_files += 1
                
                file_time = time.time() - file_start_time
                print(f"  - Processed {n_segs} segments: Stack time: {stack_time:.2f}s, "
                      f"Split time: {split_time:.2f}s, Total time: {file_time:.2f}s")
                
                # Store timing stats
                timing_stats.append({
                    "high_file": high_file,
                    "low_file": low_file,
                    "processing_time": int(file_time * 1000),
                    "file_size_high": file_size_high,
                    "file_size_low": file_size_low,
                    "is_valid": 1
                })
                
            except Exception as e:
                file_time = time.time() - file_start_time
                print(f"  - Error processing file pair {i}: {e} - Time: {file_time:.2f}s")
                
                # Try to get file sizes if possible
                try:
                    file_size_high = os.path.getsize(main_folder + 'High/' + high_file)
                    file_size_low = os.path.getsize(main_folder + 'Low/' + low_file)
                except:
                    file_size_high = 0
                    file_size_low = 0
                    
                timing_stats.append({
                    "high_file": high_file,
                    "low_file": low_file,
                    "processing_time": int(file_time * 1000),
                    "file_size_high": file_size_high,
                    "file_size_low": file_size_low,
                    "is_valid": 0
                })
    
    # Process timing stats
    print(f"\nProcessing time summary:")
    total_valid = successful_files
    total_files = len(high_freq_files)
    
    print(f"Successfully processed {total_valid}/{total_files} file pairs")
    
    # Sort by processing time to show slowest files
    timing_stats.sort(key=lambda x: x["processing_time"], reverse=True)
    
    print(f"\nTop 5 slowest files:")
    for i in range(min(5, len(timing_stats))):
        row = timing_stats[i]
        print(f"  {row['high_file']} & {row['low_file']}: {row['processing_time']/1000:.2f}s "
              f"({row['file_size_high']/1024:.1f}KB, {row['file_size_low']/1024:.1f}KB)")
    
    # Convert to numpy arrays
    print(f"\nConverting to numpy arrays...")
    convert_start = time.time()
    
    Xs_arr = np.array(all_features)
    ys_arr = np.array(all_y)
    y4s_arr = np.array(all_y4)
    y10s_arr = np.array(all_y10)
    
    convert_time = time.time() - convert_start
    print(f"Array conversion completed in {convert_time:.2f}s")
    
    total_time = time.time() - start_time_total
    print(f"\nTotal processing time: {total_time:.2f}s")
    print(f"Final arrays - Xs: {Xs_arr.shape}, ys: {ys_arr.shape}, y4s: {y4s_arr.shape}, y10s: {y10s_arr.shape}")
    
    return Xs_arr, ys_arr, y4s_arr, y10s_arr

In [6]:
# Dataset Info
main_folder = './Data/DroneRF/'
t_seg = 20
Xs_arr, ys_arr, y4s_arr, y10s_arr = load_dronerf_raw(main_folder, t_seg)
fs = 40e6 #40 MHz

print('length of X:', len(Xs_arr), 'length of y:', len(ys_arr))

Started processing at 16:19:42
Found 2 high frequency files and 2 low frequency files

Processing batch 1/1 (files 1-2)
Processing file 1/2: 10011H_0.csv (92069.7KB) & 10011L_0.csv (94588.4KB)
  - Load time: 392.39s
  - Processed 12 segments: Stack time: 0.05s, Split time: 0.00s, Total time: 392.45s
Processing file 2/2: 11000H_0.csv (93690.4KB) & 11000L_0.csv (97027.5KB)
  - Load time: 376.50s
  - Processed 12 segments: Stack time: 0.05s, Split time: 0.00s, Total time: 376.54s

Processing time summary:
Successfully processed 2/2 file pairs

Top 5 slowest files:
  10011H_0.csv & 10011L_0.csv: 392.45s (92069.7KB, 94588.4KB)
  11000H_0.csv & 11000L_0.csv: 376.54s (93690.4KB, 97027.5KB)

Converting to numpy arrays...
Array conversion completed in 0.08s

Total processing time: 769.07s
Final arrays - Xs: (24, 2, 800000), ys: (24,), y4s: (24,), y10s: (24,)
length of X: 24 length of y: 24


In [24]:
n_per_seg = 1024 # length of each segment (powers of 2)
n_overlap_spec = 120
win_type = 'hamming' # make ends of each segment match
high_low = 'H' #'L', 'H' # high or low range of frequency
feature_to_save = ['PSD'] # what features to generate and save: SPEC or PSD
format_to_save = ['ARR'] # IMG or ARR or RAW
to_add = True
spec_han_window = np.hanning(n_per_seg)

# Image properties
dim_px = (224, 224) # dimension of image pixels
dpi = 100

# Raw input len
v_samp_len = 10000

# data saving folders
features_folder = dronerf_feat_path
date_string = date.today()
# folder naming: ARR_FEAT_NFFT_SAMPLELENGTH
arr_spec_folder = "ARR_SPEC_"+high_low+'_'+str(n_per_seg)+"_"+str(t_seg)+"/"
arr_psd_folder = "ARR_PSD_"+high_low+'_'+str(n_per_seg)+"_"+str(t_seg)+"/"
img_spec_folder = "IMG_SPEC_"+high_low+'_'+str(n_per_seg)+"_"+str(t_seg)+"/"
img_psd_folder = "IMG_PSD_"+high_low+'_'+str(n_per_seg)+"_"+str(t_seg)+"/"
raw_folder = 'RAW_VOLT_'+str(v_samp_len)+"_"+str(t_seg)+"/" # high and low frequency stacked together

existing_folders = os.listdir(features_folder)

if high_low == 'H':
    i_hl = 0
elif high_low == 'L':
    i_hl = 1

In [25]:
# check if this set of parameters already exists
# check if each of the 4 folders exist
sa_save = False   #spec array
si_save = False   #spec imag
pa_save = False   #psd array
pi_save = False   #psd imag
raw_save = False # raw high low signals

if 'SPEC' in feature_to_save:
    if 'ARR' in format_to_save:
        if arr_spec_folder not in existing_folders or to_add:
            try:
                os.mkdir(features_folder+arr_spec_folder)
            except:
                print('folder already exist - adding')
            sa_save = True
            print('Generating SPEC in ARRAY format')
        else:
            print('Spec Arr folder already exists')
    if 'IMG' in format_to_save:
        if img_spec_folder not in existing_folders or to_add:
            try:
                os.mkdir(features_folder+img_spec_folder)
            except:
                print('folder already exist - adding')
            si_save = True
            print('Generating SPEC in IMAGE format')
        else:
            print('Spec Arr folder already exists')
if 'PSD' in feature_to_save:
    if 'ARR' in format_to_save:
        if arr_psd_folder not in existing_folders or to_add:
            try:
                os.mkdir(features_folder+arr_psd_folder)
            except:
                print('folder already exist - adding')
            pa_save = True
            print('Generating PSD in ARRAY format')
        else:
            print('PSD Arr folder already exists')
    if 'IMG' in format_to_save:
        if img_psd_folder not in existing_folders or to_add:
            try:
                os.mkdir(features_folder+img_psd_folder)
            except:
                print('folder already exist - adding')
            pi_save = True
            print('Generating PSD in IMAGE format')
        else:
            print('PSD Arr folder already exists')

if 'RAW' in feature_to_save:
    if raw_folder in existing_folders or to_add:
        try:
            os.mkdir(features_folder+raw_folder)
        except:
            print('RAW V folder already exists')
        raw_save = True



Generating PSD in ARRAY format


In [26]:
if all([not sa_save, not si_save, not pa_save, not pi_save, not raw_save]):
    print('Features Already Exist - Do Not Generate')
else:
    n_parts = 24 # process the data in 10 parts so memory doesn't overwhelm

    indices_ranges = np.split(np.array(range(len(Xs_arr))), n_parts) 
    for i in range(n_parts):
        BILABEL = []
        DRONELABEL = []
        MODELALBEL = []
        F_PSD = []
        F_SPEC = []
        F_V = []
        ir = indices_ranges[i]
        for j in tqdm(range(len(ir))):
            d_real = Xs_arr[ir[j]][i_hl]
            
            # if save raw data
            if raw_save:
                t = np.arange(0, len(d_real))
                f_high = interpolate.interp1d(t, Xs_arr[ir[j]][0])
                f_low = interpolate.interp1d(t, Xs_arr[ir[j]][1])
                tt = np.linspace(0, len(d_real)-1, num=v_samp_len)

                d_v = np.stack((f_high(tt), f_low(tt)), axis=0)
                F_V.append(d_v)
            
            if pa_save or pi_save:
            # calculate PSD
                fpsd, Pxx_den = signal.welch(d_real, fs, window=win_type, nperseg=n_per_seg)
                if pa_save:
                    F_PSD.append(Pxx_den)
                if pi_save:
                    save_psd_image_rf(features_folder, img_psd_folder,
                                      y10s_arr[ir[j]], i, j, Pxx_den, dim_px, dpi)
            
            if sa_save or si_save:
            # calculate spectrogram
            # welch's method older
#           fspec, t, Sxx = signal.spectrogram(d_real, fs, window=win_type, nperseg=n_per_seg)
            
                if si_save: # set up fig properties if saving images
                    plt.clf()
                    fig,ax = plt.subplots(1, figsize=(dim_px[0]/dpi, dim_px[1]/dpi), dpi=dpi)
                    fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
                    ax.axis('tight')
                    ax.axis('off')

                spec, _, _, _ = plt.specgram(d_real, NFFT=n_per_seg, Fs=fs, window=spec_han_window, 
                                  noverlap=n_overlap_spec, sides='onesided')
                if si_save:
                    save_spec_image_fig_rf(features_folder, img_spec_folder, 
                                           y10s_arr[ir[j]], i, j, fig, dpi)
                if sa_save:
                    F_SPEC.append(interpolate_2d(Sxx, (224,224)))

            # Labels
            BILABEL.append(ys_arr[ir[j]])
            DRONELABEL.append(y4s_arr[ir[j]])
            MODELALBEL.append(y10s_arr[ir[j]])
        
        if sa_save:
            save_array_rf(features_folder+arr_spec_folder, F_SPEC, BILABEL, DRONELABEL, MODELALBEL, 'SPEC', n_per_seg, i)
        if pa_save:
            save_array_rf(features_folder+arr_psd_folder, F_PSD, BILABEL, DRONELABEL, MODELALBEL, 'PSD', n_per_seg, i)
        if raw_save:
            save_array_rf(features_folder+raw_folder, F_V, BILABEL, DRONELABEL, MODELALBEL, 'RAW', '', i)

100%|██████████| 1/1 [00:00<00:00, 23.47it/s]


Features_DroneRF/ARR_PSD_H_1024_20/PSD_1024_0


100%|██████████| 1/1 [00:00<00:00, 21.74it/s]


Features_DroneRF/ARR_PSD_H_1024_20/PSD_1024_1


100%|██████████| 1/1 [00:00<00:00, 19.17it/s]


...

Features_DroneRF/ARR_PSD_H_1024_20/PSD_1024_21


100%|██████████| 1/1 [00:00<00:00, 19.79it/s]


Features_DroneRF/ARR_PSD_H_1024_20/PSD_1024_22


100%|██████████| 1/1 [00:00<00:00, 10.23it/s]

Features_DroneRF/ARR_PSD_H_1024_20/PSD_1024_23


In [ ]:
import pandas as pd
df = pd.read_csv('Data\\DroneRF\\High\\10011H_0.csv')  # `.compute()` converts to pandas DataFrame

In [ ]:
df.head()

ModuleNotFoundError: No module named 'ray'

In [ ]:
df.head()